# Vector Stores Tutorial: Qdrant & Weaviate

## 🎯 Learning Objectives

In this tutorial, you'll learn:
- What vector stores are and why they're essential for RAG (Retrieval-Augmented Generation)
- How to work with **Qdrant** (local, in-memory, and persistent storage)
- How to work with **Weaviate** (Docker-based setup)
- How to use **Ollama embeddings** for real semantic search
- Metadata filtering with different vector store syntaxes
- When to use each vector store based on your use case

## 📚 What are Vector Stores?

Vector stores (also called vector databases) are specialized databases that:
1. Store **embeddings** - numerical representations of text
2. Enable **semantic search** - finding similar content based on meaning, not just keywords
3. Power **RAG systems** - providing relevant context to LLMs

## 📋 Prerequisites

Before starting, you should have:
- Python 3.8+ installed
- Basic understanding of LangChain
- Ollama installed and running (for real embeddings)
- Docker (optional, for Weaviate)

---

## 📦 Installation & Setup

### Step 1: Install Python Packages

Run these commands in your terminal:

```bash
# For Qdrant
pip install langchain-qdrant qdrant-client

# For Weaviate
pip install langchain-weaviate weaviate-client

# For Ollama embeddings
pip install langchain-ollama

# Core LangChain
pip install langchain-core
```

### Step 2: Set Up Ollama (for Real Embeddings)

```bash
# Install Ollama from https://ollama.ai

# Start Ollama service
ollama serve

# Pull the embedding model (in another terminal)
ollama pull nomic-embed-text
```

### Step 3: Set Up Weaviate (Optional - Only for Weaviate Section)

```bash
# Run Weaviate in Docker
docker run -d -p 8080:8080 -p 50051:50051 \
  --name weaviate \
  cr.weaviate.io/semitechnologies/weaviate:latest
```

### 📌 Note
If you don't have Ollama set up, the code will show you error messages with instructions. You can still run the Qdrant sections to learn the concepts!

---

## 🔧 Imports and Setup

### Important: Correct Imports for LangChain 1.0+

LangChain 1.0+ has reorganized its modules. Here's what you need to know:

✅ **CORRECT:**
```python
from langchain_core.documents import Document
```

❌ **DEPRECATED (will cause errors):**
```python
from langchain.schema import Document  # Old way
from langchain_core.memory import ...  # Memory not in langchain_core
```

💡 **For memory/chat history, use:**
```python
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
```

In [1]:
# Standard library imports
import os
import sys

# LangChain core - Document class
from langchain_core.documents import Document

# Qdrant imports
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, Filter, FieldCondition, MatchValue

# Ollama embeddings
from langchain_ollama import OllamaEmbeddings

print("✓ All imports loaded correctly!")
print("✓ Using langchain_core.documents.Document (correct LangChain 1.0+ import)")

✓ All imports loaded correctly!
✓ Using langchain_core.documents.Document (correct LangChain 1.0+ import)


## 🧠 Understanding Embeddings

### What are Embeddings?

**Embeddings** are numerical representations of text that capture semantic meaning. Think of them as coordinates in a high-dimensional space where similar meanings are close together.

**Example:**
- "cat" and "kitten" → Similar embeddings (close in vector space)
- "cat" and "astronomy" → Different embeddings (far apart)

### The nomic-embed-text Model

We're using **nomic-embed-text** from Ollama:
- **Dimension:** 768 (each text becomes a list of 768 numbers)
- **Purpose:** Optimized for text retrieval and semantic search
- **Speed:** Fast enough for real-time applications

### How Embeddings Power Search

1. **Storage:** Convert documents → embeddings → store in vector database
2. **Query:** Convert your question → embedding
3. **Search:** Find stored embeddings closest to your query embedding
4. **Retrieve:** Return the original documents

---

In [2]:
# Initialize Ollama embeddings
# This connects to your local Ollama service and uses the nomic-embed-text model

print("Initializing Ollama embeddings (nomic-embed-text)...")
print("Make sure Ollama is running: 'ollama serve'\n")

embeddings = OllamaEmbeddings(model="nomic-embed-text")

print("✓ Ollama embeddings initialized")
print("  Model: nomic-embed-text")
print("  Dimension: 768")

Initializing Ollama embeddings (nomic-embed-text)...
Make sure Ollama is running: 'ollama serve'

✓ Ollama embeddings initialized
  Model: nomic-embed-text
  Dimension: 768


## 📄 Creating Sample Documents

Let's create some sample documents to work with. Each document has:
- **page_content:** The actual text content
- **metadata:** Additional information (topic, difficulty level, etc.)

Metadata is useful for filtering - you can search for documents matching specific criteria!

In [3]:
# Create sample documents with metadata
# The Document class comes from langchain_core.documents

sample_docs = [
    Document(
        page_content="RAG combines retrieval and generation",
        metadata={"topic": "rag", "difficulty": "intermediate"}
    ),
    Document(
        page_content="LangChain simplifies LLM applications",
        metadata={"topic": "langchain", "difficulty": "beginner"}
    ),
    Document(
        page_content="Vector databases enable semantic search",
        metadata={"topic": "vectordb", "difficulty": "intermediate"}
    )
]

print("✓ Created 3 sample documents:")
for i, doc in enumerate(sample_docs, 1):
    print(f"  {i}. {doc.page_content}")
    print(f"     Metadata: {doc.metadata}")

✓ Created 3 sample documents:
  1. RAG combines retrieval and generation
     Metadata: {'topic': 'rag', 'difficulty': 'intermediate'}
  2. LangChain simplifies LLM applications
     Metadata: {'topic': 'langchain', 'difficulty': 'beginner'}
  3. Vector databases enable semantic search
     Metadata: {'topic': 'vectordb', 'difficulty': 'intermediate'}


---

# 🗄️ Part 1: Qdrant Vector Store

## What is Qdrant?

**Qdrant** is a vector database that excels at:
- ✅ Local development (no Docker required)
- ✅ Fast similarity search
- ✅ Flexible storage options (in-memory or persistent)
- ✅ Rich metadata filtering

## Three Ways to Use Qdrant

We'll explore three approaches:

1. **In-Memory** - Fast but temporary (lost when script ends)
2. **Persistent** - Saved to disk (survives restarts)
3. **from_documents** - Easiest method (recommended)

## 🔍 Filter Syntax Reference

Different vector stores use different filter syntax:

**ChromaDB (simple dict):**
```python
filter={"topic": "rag"}
```

**Qdrant (Filter object):**
```python
from qdrant_client.models import Filter, FieldCondition, MatchValue
filter=Filter(
    must=[
        FieldCondition(
            key="metadata.topic",
            match=MatchValue(value="rag")
        )
    ]
)
```

---

## Option 1: Qdrant In-Memory (No Persistence)

### When to Use
- Quick testing and experiments
- Temporary data that doesn't need to be saved
- Learning and prototyping

### How It Works
1. Create an in-memory client with `location=":memory:"`
2. Create a collection with vector configuration (size=768, distance=COSINE)
3. Add documents and search
4. **Data is lost when the program ends**

In [4]:
print("=" * 80)
print("QDRANT IN-MEMORY EXAMPLE")
print("=" * 80)
print()

# Step 1: Create in-memory Qdrant client
# The `:memory:` location means data is stored in RAM (not saved to disk)
qdrant_client_memory = QdrantClient(location=":memory:")

# Step 2: Create a collection
# - collection_name: identifier for this collection
# - size: must match embedding dimension (768 for nomic-embed-text)
# - distance: COSINE measures similarity (other options: DOT, EUCLID)
qdrant_client_memory.recreate_collection(
    collection_name="my_collection_memory",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

# Step 3: Create Qdrant vector store wrapper
# This LangChain wrapper makes it easy to work with Qdrant
qdrant_store_memory = QdrantVectorStore(
    client=qdrant_client_memory,
    collection_name="my_collection_memory",
    embedding=embeddings
)

# Step 4: Add documents to the store
# This will automatically:
# 1. Convert documents to embeddings using Ollama
# 2. Store embeddings + metadata in Qdrant
qdrant_store_memory.add_documents(sample_docs)
print("✓ Added documents to Qdrant (in-memory)")
print("  Collection: my_collection_memory")
print("  Documents: 3")
print("  Storage: RAM (temporary)")

QDRANT IN-MEMORY EXAMPLE



/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_7541/3668306292.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client_memory.recreate_collection(


✓ Added documents to Qdrant (in-memory)
  Collection: my_collection_memory
  Documents: 3
  Storage: RAM (temporary)


### Basic Similarity Search

Let's search for documents similar to our query. The vector store will:
1. Convert your query to an embedding
2. Find the k most similar document embeddings
3. Return the original documents

sample_docs = [
    Document(
        page_content="RAG combines retrieval and generation",
        metadata={"topic": "rag", "difficulty": "intermediate"}
    ),
    Document(
        page_content="LangChain simplifies LLM applications",
        metadata={"topic": "langchain", "difficulty": "beginner"}
    ),
    Document(
        page_content="Vector databases enable semantic search",
        metadata={"topic": "vectordb", "difficulty": "intermediate"}
    )
]

In [5]:
print("\n" + "-" * 80)
print("BASIC SIMILARITY SEARCH")
print("-" * 80)

# Search for documents similar to this query
# k=2 means return the top 2 most similar documents
results = qdrant_store_memory.similarity_search(
    "Tell me about RAG",
    k=2
)

print("\nQuery: 'Tell me about RAG'")
print("\nSearch results:")
for i, doc in enumerate(results, 1):
    print(f"  {i}. {doc.page_content}")
    print(f"     Metadata: {doc.metadata}")

print("\n💡 Notice: The document about 'RAG combines retrieval...' is returned first")
print("   because it's semantically most similar to our query!")


--------------------------------------------------------------------------------
BASIC SIMILARITY SEARCH
--------------------------------------------------------------------------------

Query: 'Tell me about RAG'

Search results:
  1. RAG combines retrieval and generation
     Metadata: {'topic': 'rag', 'difficulty': 'intermediate', '_id': '1e497c35b5d042a9bfc8b5a348efa090', '_collection_name': 'my_collection_memory'}
  2. Vector databases enable semantic search
     Metadata: {'topic': 'vectordb', 'difficulty': 'intermediate', '_id': '31dfd37d672744e1ae66339a057512b9', '_collection_name': 'my_collection_memory'}

💡 Notice: The document about 'RAG combines retrieval...' is returned first
   because it's semantically most similar to our query!


### Search with Metadata Filtering

Sometimes you want to search within a subset of documents. Qdrant uses a **Filter object** for this.

#### Filter Structure
```python
Filter(
    must=[...],   # All conditions must match (AND logic)
    should=[...], # At least one condition must match (OR logic)
    must_not=[...] # No conditions should match (NOT logic)
)
```

In [8]:
print("\n" + "-" * 80)
print("SEARCH WITH METADATA FILTER")
print("-" * 80)
#metadata={"topic": ["rag", "llms", "agents"]}
# Create a filter to only search documents with topic='rag'
# Note: We use 'metadata.topic' because metadata is nested
qdrant_filter = Filter(
    must=[
        FieldCondition(
            key="metadata.topic",
            match=MatchValue(value="rag")
        )
    ]
)

# Same search, but only among filtered documents
results_filtered = qdrant_store_memory.similarity_search(
    "Tell me about RAG",
    k=2,
    filter=qdrant_filter
)

print("\nQuery: 'Tell me about RAG'")
print("Filter: topic='rag'")
print("\nFiltered search results:")
for i, doc in enumerate(results_filtered, 1):
    print(f"  {i}. {doc.page_content}")
    print(f"     Metadata: {doc.metadata}")

print("\n💡 Only documents with topic='rag' are returned!")


--------------------------------------------------------------------------------
SEARCH WITH METADATA FILTER
--------------------------------------------------------------------------------

Query: 'Tell me about RAG'
Filter: topic='rag'

Filtered search results:
  1. RAG combines retrieval and generation
     Metadata: {'topic': 'rag', 'difficulty': 'intermediate', '_id': '1e497c35b5d042a9bfc8b5a348efa090', '_collection_name': 'my_collection_memory'}

💡 Only documents with topic='rag' are returned!


### Multiple Filter Conditions (AND Logic)

You can combine multiple conditions. All conditions in the `must` list must match.

In [7]:
print("\n" + "-" * 80)
print("MULTIPLE FILTERS EXAMPLE (AND LOGIC)")
print("-" * 80)

# Filter for documents where:
# - topic='rag' AND
# - difficulty='intermediate'
multi_filter = Filter(
    must=[
        FieldCondition(key="metadata.topic", match=MatchValue(value="rag")),
        FieldCondition(key="metadata.difficulty", match=MatchValue(value="intermediate"))
    ]
)

# You can use this filter in similarity_search:
# results_multi = qdrant_store_memory.similarity_search("RAG", k=2, filter=multi_filter)

print("\nFilter structure:")
print("  must=[")
print("    FieldCondition(key='metadata.topic', match='rag'),")
print("    FieldCondition(key='metadata.difficulty', match='intermediate')")
print("  ]")
print("\n💡 Both conditions must be true (AND logic)")
print("💡 For OR logic, use should=[...] instead of must=[...]")


--------------------------------------------------------------------------------
MULTIPLE FILTERS EXAMPLE (AND LOGIC)
--------------------------------------------------------------------------------

Filter structure:
  must=[
    FieldCondition(key='metadata.topic', match='rag'),
    FieldCondition(key='metadata.difficulty', match='intermediate')
  ]

💡 Both conditions must be true (AND logic)
💡 For OR logic, use should=[...] instead of must=[...]


In [9]:
# Same search, but only among filtered documents
results_filtered = qdrant_store_memory.similarity_search(
    "Tell me about RAG",
    k=2,
    filter=multi_filter
)

print("\nQuery: 'Tell me about RAG'")
print("Filter: topic='rag'")
print("\nFiltered search results:")
for i, doc in enumerate(results_filtered, 1):
    print(f"  {i}. {doc.page_content}")
    print(f"     Metadata: {doc.metadata}")

print("\n💡 Only documents with topic='rag' are returned!")


Query: 'Tell me about RAG'
Filter: topic='rag'

Filtered search results:
  1. RAG combines retrieval and generation
     Metadata: {'topic': 'rag', 'difficulty': 'intermediate', '_id': '8db7a2692a594a358114d84e832b3f58', '_collection_name': 'my_collection_memory'}

💡 Only documents with topic='rag' are returned!


---

## Option 2: Qdrant with Local Persistence

### When to Use
- Data that needs to survive program restarts
- Building a document index you'll reuse
- Development/testing with persistent data

### How It Works
1. Create a client with a file path (e.g., `"./qdrant_data"`)
2. Qdrant stores data in that directory
3. Data persists between runs
4. Can be version controlled (just commit the directory)

In [9]:
print("\n" + "=" * 80)
print("QDRANT WITH LOCAL PERSISTENCE")
print("=" * 80)
print()

# Step 1: Specify a local directory for storage
qdrant_path = "./qdrant_data"

# Step 2: Create persistent Qdrant client
# Data will be saved in the ./qdrant_data directory
qdrant_client_persistent = QdrantClient(path=qdrant_path)

# Step 3: Create collection (same as before)
qdrant_client_persistent.recreate_collection(
    collection_name="my_collection_persistent",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

# Step 4: Create vector store wrapper
qdrant_store_persistent = QdrantVectorStore(
    client=qdrant_client_persistent,
    collection_name="my_collection_persistent",
    embedding=embeddings
)

# Step 5: Add documents
qdrant_store_persistent.add_documents(sample_docs)
print(f"✓ Added documents to Qdrant (persistent)")
print(f"  Storage location: {qdrant_path}")
print(f"  Collection: my_collection_persistent")
print(f"  ⚠️  Data will persist even after this script ends!")

# Step 6: Search
results = qdrant_store_persistent.similarity_search(
    "Tell me about LangChain",
    k=2
)

print("\nQuery: 'Tell me about LangChain'")
print("\nSearch results:")
for i, doc in enumerate(results, 1):
    print(f"  {i}. {doc.page_content}")
    print(f"     Metadata: {doc.metadata}")

print("\n💡 Next time you run this, you can load the same data from disk!")


QDRANT WITH LOCAL PERSISTENCE



/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_7541/1087051481.py:14: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client_persistent.recreate_collection(


✓ Added documents to Qdrant (persistent)
  Storage location: ./qdrant_data
  Collection: my_collection_persistent
  ⚠️  Data will persist even after this script ends!

Query: 'Tell me about LangChain'

Search results:
  1. LangChain simplifies LLM applications
     Metadata: {'topic': 'langchain', 'difficulty': 'beginner', '_id': 'deb330d94d3441a08c29ab842159c57c', '_collection_name': 'my_collection_persistent'}
  2. RAG combines retrieval and generation
     Metadata: {'topic': 'rag', 'difficulty': 'intermediate', '_id': 'e2dcbc1368464b3499e6232e4e177950', '_collection_name': 'my_collection_persistent'}

💡 Next time you run this, you can load the same data from disk!


---

## Option 3: Qdrant from_documents (Recommended ⭐)

### Why This is Recommended
- ✅ Simplest syntax (one method call)
- ✅ Handles collection creation automatically
- ✅ Supports persistence with just a `path` parameter
- ✅ Returns a ready-to-use vector store

### When to Use
- Starting a new project
- Quick prototyping
- Most common use cases

In [10]:
print("\n" + "=" * 80)
print("QDRANT FROM_DOCUMENTS (RECOMMENDED METHOD)")
print("=" * 80)
print()

# Create Qdrant store directly from documents
# This is the easiest way - everything happens in one call!
qdrant_store_easy = QdrantVectorStore.from_documents(
    documents=sample_docs,          # Your documents
    embedding=embeddings,            # Embedding function
    path="./qdrant_easy",           # Local persistence (optional)
    collection_name="rag_collection" # Collection name
)

print("✓ Created Qdrant store from documents")
print("  Collection: rag_collection")
print("  Storage: ./qdrant_easy")
print("  Documents: 3")
print("\n💡 This is the recommended approach for most use cases!")


QDRANT FROM_DOCUMENTS (RECOMMENDED METHOD)

✓ Created Qdrant store from documents
  Collection: rag_collection
  Storage: ./qdrant_easy
  Documents: 3

💡 This is the recommended approach for most use cases!


### Similarity Search with Scores

Sometimes you want to know **how similar** each result is. Use `similarity_search_with_score()` to get scores.

**Score Interpretation:**
- **Higher scores** = More similar
- **COSINE distance:** Ranges from -1 to 1 (1 = identical, 0 = orthogonal, -1 = opposite)
- In practice, scores > 0.7 are considered very similar

In [11]:
# Search with scores
results_with_scores = qdrant_store_easy.similarity_search_with_score(
    "Vector databases",
    k=3
)

print("\nQuery: 'Vector databases'")
print("\nSearch results with similarity scores:")
print()
for doc, score in results_with_scores:
    print(f"  Score: {score:.4f}")  # Similarity score (higher = more similar)
    print(f"  Content: {doc.page_content}")
    print(f"  Metadata: {doc.metadata}")
    print()

print("💡 Scores help you filter out low-quality results")
print("💡 You can set a threshold (e.g., only return results with score > 0.7)")


Query: 'Vector databases'

Search results with similarity scores:

  Score: 0.7916
  Content: Vector databases enable semantic search
  Metadata: {'topic': 'vectordb', 'difficulty': 'intermediate', '_id': 'fd3d3c8ab5bc4a0d9bb4aebb2f8e7f3c', '_collection_name': 'rag_collection'}

  Score: 0.4856
  Content: RAG combines retrieval and generation
  Metadata: {'topic': 'rag', 'difficulty': 'intermediate', '_id': 'fcd1001bd422475980aa17c850e9d6a2', '_collection_name': 'rag_collection'}

  Score: 0.3989
  Content: LangChain simplifies LLM applications
  Metadata: {'topic': 'langchain', 'difficulty': 'beginner', '_id': '662c642375064d288bc7e69967647674', '_collection_name': 'rag_collection'}

💡 Scores help you filter out low-quality results
💡 You can set a threshold (e.g., only return results with score > 0.7)


---

# 🗄️ Part 2: Weaviate Vector Store

## What is Weaviate?

**Weaviate** is a production-ready vector database that offers:
- ✅ Excellent scalability (handles millions of vectors)
- ✅ Advanced features (hybrid search, multi-tenancy)
- ✅ GraphQL API
- ✅ Built for production environments

## When to Use Weaviate

✅ **Use Weaviate when:**
- Building production applications
- Need to scale to large datasets (millions of documents)
- Want advanced features like hybrid search
- Have Docker available

❌ **Don't use Weaviate when:**
- Doing quick prototypes (use Qdrant instead)
- Can't use Docker
- Working with small datasets (< 10k documents)

## Setup Requirements

Weaviate requires Docker to run locally:
```bash
docker run -d -p 8080:8080 -p 50051:50051 \
  --name weaviate \
  cr.weaviate.io/semitechnologies/weaviate:latest
```

## Filter Syntax Difference

Weaviate uses `where_filter` with a different syntax than Qdrant:

```python
where_filter={
    "path": ["difficulty"],
    "operator": "Equal",
    "valueText": "intermediate"
}
```

---

In [14]:
from weaviate.classes.query import Filter


from weaviate.classes.query import Filter

results_filtered = weaviate_store.similarity_search(
        "Tell me about databases",
        k=2,
        filters=Filter.by_property("difficulty").equal("intermediate")  # Proper Filter object
    )

In [15]:
from weaviate.classes.query import Filter


print("\n" + "=" * 80)
print("WEAVIATE LOCAL VECTOR STORE EXAMPLE")
print("=" * 80)
print()
print("⚠️  Note: This requires Weaviate running locally on port 8080")
print("   If not running, you'll see connection errors (that's OK for learning!)")
print()

try:
    import weaviate
    from langchain_weaviate import WeaviateVectorStore
    
    print("-" * 80)
    print("Connecting to Local Weaviate")
    print("-" * 80)
    
    # Step 1: Connect to local Weaviate instance
    # This assumes Weaviate is running on localhost:8080
    weaviate_client = weaviate.connect_to_local(
        host="localhost",
        port=8080,
        grpc_port=50051
    )
    
    print("✓ Connected to local Weaviate")
    print("  Host: localhost:8080")
    print("  gRPC Port: 50051")
    
    # Step 2: Create Weaviate vector store
    print("\n" + "-" * 80)
    print("Creating Weaviate Vector Store")
    print("-" * 80)
    
    weaviate_store = WeaviateVectorStore(
        client=weaviate_client,
        index_name="MyDocuments",  # Collection name in Weaviate
        text_key="text",            # Field name for document text
        embedding=embeddings
    )
    
    # Step 3: Add documents
    weaviate_store.add_documents(sample_docs)
    print("✓ Added documents to Weaviate")
    print("  Index: MyDocuments")
    print("  Documents: 3")
    
    # Step 4: Basic Search
    print("\n" + "-" * 80)
    print("Basic Search")
    print("-" * 80)
    
    results = weaviate_store.similarity_search(
        "Tell me about RAG",
        k=2
    )
    
    print("\nQuery: 'Tell me about RAG'")
    print("\nSearch results:")
    for i, doc in enumerate(results, 1):
        print(f"  {i}. {doc.page_content}")
        print(f"     Metadata: {doc.metadata}")
    
    # Step 5: Search with Metadata Filter
    print("\n" + "-" * 80)
    print("Search with Metadata Filter")
    print("-" * 80)
    
    # Weaviate uses where_filter with different syntax
    results_filtered = weaviate_store.similarity_search("Tell me about databases", 
    k=2, 
    filters=Filter.by_property("difficulty").equal("intermediate") ) # Proper Filter object
    
    print("\nQuery: 'Tell me about databases'")
    print("Filter: difficulty='intermediate'")
    print("\nFiltered search results:")
    for i, doc in enumerate(results_filtered, 1):
        print(f"  {i}. {doc.page_content}")
        print(f"     Metadata: {doc.metadata}")
    
    # Step 6: Search with Scores
    print("\n" + "-" * 80)
    print("Search with Scores")
    print("-" * 80)
    
    results_with_scores = weaviate_store.similarity_search_with_score(
        "Vector databases",
        k=3
    )
    
    print("\nQuery: 'Vector databases'")
    print("\nSearch results with scores:")
    for doc, score in results_with_scores:
        print(f"  Score: {score:.4f}")
        print(f"  Content: {doc.page_content}")
        print(f"  Metadata: {doc.metadata}")
        print()
    
    # Step 7: Alternative - Create from Documents
    print("-" * 80)
    print("Creating Weaviate from Documents (Alternative Method)")
    print("-" * 80)
    
    weaviate_store_easy = WeaviateVectorStore.from_documents(
        documents=sample_docs,
        embedding=embeddings,
        client=weaviate_client,
        index_name="EasyDocuments"
    )
    
    print("✓ Created Weaviate store from documents")
    
    # Quick search
    results = weaviate_store_easy.similarity_search("LangChain", k=2)
    print("\nQuick search results:")
    for i, doc in enumerate(results, 1):
        print(f"  {i}. {doc.page_content}")
    
    # Clean up
    weaviate_client.close()
    print("\n✓ Closed Weaviate connection")
    
except Exception as e:
    print(f"✗ Weaviate error: {e}")
    print()
    print("Troubleshooting:")
    print("1. Check if Weaviate is running: docker ps")
    print("2. Start Weaviate: docker run -d -p 8080:8080 -p 50051:50051 \\")
    print("     --name weaviate cr.weaviate.io/semitechnologies/weaviate:latest")
    print("3. Check if port 8080 is available: lsof -i :8080")
    print("4. Check Weaviate logs: docker logs weaviate")
    print()
    print("💡 It's OK if this doesn't work - you can still learn from the code!")


WEAVIATE LOCAL VECTOR STORE EXAMPLE

⚠️  Note: This requires Weaviate running locally on port 8080
   If not running, you'll see connection errors (that's OK for learning!)

--------------------------------------------------------------------------------
Connecting to Local Weaviate
--------------------------------------------------------------------------------
✓ Connected to local Weaviate
  Host: localhost:8080
  gRPC Port: 50051

--------------------------------------------------------------------------------
Creating Weaviate Vector Store
--------------------------------------------------------------------------------


/Users/sourangshupal/Downloads/simple-rag-langchain/.venv/lib/python3.13/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/var/folders/cj/13vbmk7n7fqgmdnqjjwn1bx80000gn/T/ipykernel_7541/2242043336.py:37: ResourceWarning: unclosed <socket.socket fd=97, family=30, type=1, proto=6, laddr=('::1', 62259, 0, 0), raddr=('::1', 8080, 0, 0)>
  weaviate_store = WeaviateVectorStore(


✓ Added documents to Weaviate
  Index: MyDocuments
  Documents: 3

--------------------------------------------------------------------------------
Basic Search
--------------------------------------------------------------------------------

Query: 'Tell me about RAG'

Search results:
  1. RAG combines retrieval and generation
     Metadata: {'difficulty': 'intermediate', 'topic': 'rag'}
  2. RAG combines retrieval and generation
     Metadata: {'difficulty': 'intermediate', 'topic': 'rag'}

--------------------------------------------------------------------------------
Search with Metadata Filter
--------------------------------------------------------------------------------

Query: 'Tell me about databases'
Filter: difficulty='intermediate'

Filtered search results:
  1. Vector databases enable semantic search
     Metadata: {'difficulty': 'intermediate', 'topic': 'vectordb'}
  2. Vector databases enable semantic search
     Metadata: {'difficulty': 'intermediate', 'topic': 'vecto

---

# 🚀 Part 3: Real Ollama Embeddings Integration

## Complete Working Example

This section demonstrates a complete, production-ready setup with:
- ✅ Real Ollama embeddings (not dummy/fake embeddings)
- ✅ Qdrant for storage
- ✅ Ready to integrate into RAG pipelines

## Testing Your Ollama Connection

First, let's verify that Ollama is working correctly.

In [12]:
print("\n" + "=" * 80)
print("BONUS: REAL OLLAMA EMBEDDINGS EXAMPLE")
print("=" * 80)
print()

try:
    from langchain_ollama import OllamaEmbeddings
    from langchain_qdrant import QdrantVectorStore
    
    print("-" * 80)
    print("Testing Ollama Connection")
    print("-" * 80)
    
    # Initialize Ollama embeddings
    ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")
    
    # Test embeddings by converting a sample text
    test_text = "This is a test with real Ollama embeddings"
    test_embedding = ollama_embeddings.embed_query(test_text)
    
    print(f"✓ Ollama embeddings working!")
    print(f"  Model: nomic-embed-text")
    print(f"  Embedding dimension: {len(test_embedding)}")
    print(f"  Sample values (first 5): {[f'{x:.4f}' for x in test_embedding[:5]]}")
    print()
    print("💡 Each document is converted to 768 numbers that capture its meaning!")
    
    # Create Qdrant store with real embeddings
    print("\n" + "-" * 80)
    print("Creating Qdrant with Ollama Embeddings")
    print("-" * 80)
    
    ollama_qdrant_store = QdrantVectorStore.from_documents(
        documents=sample_docs,
        embedding=ollama_embeddings,
        path="./qdrant_ollama",
        collection_name="ollama_collection"
    )
    
    print("✓ Created Qdrant store with Ollama embeddings")
    print("  Storage: ./qdrant_ollama")
    print("  Collection: ollama_collection")
    
    # Search with real embeddings
    print("\n" + "-" * 80)
    print("Searching with Real Embeddings")
    print("-" * 80)
    
    query = "Tell me about RAG"
    results = ollama_qdrant_store.similarity_search(query, k=2)
    
    print(f"\nQuery: '{query}'")
    print("\nResults:")
    for i, doc in enumerate(results, 1):
        print(f"  {i}. {doc.page_content}")
        print(f"     Topic: {doc.metadata.get('topic')}")
        print(f"     Difficulty: {doc.metadata.get('difficulty')}")
    
    print("\n✓ Ollama integration successful!")
    
    # Show how to use in RAG
    print("\n" + "-" * 80)
    print("Using in RAG Pipeline")
    print("-" * 80)
    print()
    print("To use this vector store in a RAG pipeline:")
    print()
    print("  # Convert to retriever")
    print("  retriever = ollama_qdrant_store.as_retriever(")
    print("      search_kwargs={'k': 4}  # Return top 4 results")
    print("  )")
    print()
    print("  # Use in RAG chain")
    print("  from langchain_core.runnables import RunnablePassthrough")
    print("  from langchain_core.output_parsers import StrOutputParser")
    print()
    print("  rag_chain = (")
    print("      {'context': retriever, 'question': RunnablePassthrough()}")
    print("      | prompt")
    print("      | llm")
    print("      | StrOutputParser()")
    print("  )")
    print()
    print("💡 The retriever fetches relevant docs, then the LLM generates an answer!")
    
except ImportError:
    print("✗ langchain-ollama not installed")
    print()
    print("Install with:")
    print("  pip install langchain-ollama")
    print()
    print("Example code to use when installed:")
    print()
    print("from langchain_ollama import OllamaEmbeddings")
    print("from langchain_qdrant import QdrantVectorStore")
    print()
    print("# Initialize embeddings")
    print("embeddings = OllamaEmbeddings(model='nomic-embed-text')")
    print()
    print("# Create vector store")
    print("qdrant_store = QdrantVectorStore.from_documents(")
    print("    documents=your_documents,")
    print("    embedding=embeddings,")
    print("    path='./qdrant_local',")
    print("    collection_name='my_collection'")
    print(")")
    print()
    print("# Use as retriever in RAG")
    print("retriever = qdrant_store.as_retriever(search_kwargs={'k': 4})")
    
except Exception as e:
    print(f"✗ Error with Ollama: {e}")
    print()
    print("Troubleshooting:")
    print("1. Make sure Ollama is running:")
    print("   ollama serve")
    print()
    print("2. Make sure the model is downloaded:")
    print("   ollama pull nomic-embed-text")
    print()
    print("3. Make sure langchain-ollama is installed:")
    print("   pip install langchain-ollama")
    print()
    print("4. Check Ollama is accessible:")
    print("   curl http://localhost:11434")


BONUS: REAL OLLAMA EMBEDDINGS EXAMPLE

--------------------------------------------------------------------------------
Testing Ollama Connection
--------------------------------------------------------------------------------
✓ Ollama embeddings working!
  Model: nomic-embed-text
  Embedding dimension: 768
  Sample values (first 5): ['0.0496', '0.0668', '-0.1677', '-0.0624', '0.0201']

💡 Each document is converted to 768 numbers that capture its meaning!

--------------------------------------------------------------------------------
Creating Qdrant with Ollama Embeddings
--------------------------------------------------------------------------------
✓ Created Qdrant store with Ollama embeddings
  Storage: ./qdrant_ollama
  Collection: ollama_collection

--------------------------------------------------------------------------------
Searching with Real Embeddings
--------------------------------------------------------------------------------

Query: 'Tell me about RAG'

Results:


---

# 📊 Vector Store Comparison

## Feature Comparison Table

| Feature | Qdrant | Weaviate | ChromaDB |
|---------|--------|----------|----------|
| **Setup** | Easy (no Docker) | Requires Docker | Easiest (pip install) |
| **Persistence** | Local file/in-memory | Docker volume | Local file/in-memory |
| **Performance** | Fast | Very fast | Fast |
| **Scalability** | Good (millions) | Excellent (billions) | Good (thousands) |
| **Filter Syntax** | Filter objects | where_filter dict | Simple dict |
| **Memory Usage** | Low | Medium | Low |
| **Production Ready** | Yes | Yes | Limited |
| **Best For** | Local development | Production apps | Quick prototypes |

## Recommendations

### Use **Qdrant** when:
- 🎯 Local development and testing
- 🎯 Don't want to manage Docker
- 🎯 Need good performance for small-to-medium datasets
- 🎯 Want file-based persistence

### Use **Weaviate** when:
- 🎯 Building production applications
- 🎯 Need to scale to large datasets (millions of vectors)
- 🎯 Want advanced features (GraphQL, hybrid search)
- 🎯 Have Docker infrastructure

### Use **ChromaDB** when:
- 🎯 Quick prototyping and demos
- 🎯 Absolute simplest setup
- 🎯 Small datasets (< 10k documents)
- 🎯 Learning vector databases

## Performance Considerations

### Dataset Size Guidelines
- **Small (< 10k docs):** Any vector store works
- **Medium (10k - 100k docs):** Qdrant or Weaviate
- **Large (100k - 1M+ docs):** Weaviate or managed services

### Memory Usage
- **In-memory:** Fast but limited by RAM
- **Persistent:** Slower but can handle larger datasets
- **Rule of thumb:** Budget ~1KB per document + embedding size

---

---

# 🎓 Summary and Next Steps

## What You've Learned

### 1. Vector Store Basics
- ✅ What vector stores are and why they're essential for RAG
- ✅ How embeddings convert text to numbers that capture meaning
- ✅ How similarity search finds relevant documents

### 2. Qdrant
- ✅ Three ways to use Qdrant (in-memory, persistent, from_documents)
- ✅ Basic similarity search
- ✅ Metadata filtering with Filter objects
- ✅ Multiple filter conditions (AND logic)
- ✅ Similarity search with scores

### 3. Weaviate
- ✅ Docker-based setup
- ✅ Different filter syntax (where_filter)
- ✅ When to use Weaviate vs Qdrant

### 4. Ollama Integration
- ✅ Real embeddings with nomic-embed-text
- ✅ Complete working example
- ✅ How to use in RAG pipelines

### 5. Best Practices
- ✅ Correct LangChain 1.0+ imports
- ✅ When to use each vector store
- ✅ How to handle errors and troubleshooting

## Key Takeaways

1. **Start with Qdrant** for local development (no Docker needed)
2. **Use `from_documents()`** for the simplest setup
3. **Always use correct imports**: `from langchain_core.documents import Document`
4. **Filter syntax varies** between vector stores (learn each one)
5. **Ollama provides free, local embeddings** (great for development)

## Next Steps

### 1. Practice Exercises
Try these on your own:

**Exercise 1:** Create your own documents
- Create 5-10 documents about a topic you know
- Add meaningful metadata
- Test different search queries

**Exercise 2:** Experiment with filters
- Create documents with multiple metadata fields
- Try different filter combinations
- Compare results with and without filters

**Exercise 3:** Compare vector stores
- Use the same documents in Qdrant and Weaviate
- Search for the same queries
- Compare the results and performance

**Exercise 4:** Build a simple RAG system
- Load documents from a file
- Create a vector store
- Build a retriever
- Integrate with an LLM

### 2. Explore More Features
- **Hybrid search** (combining keyword and semantic search)
- **MMR (Maximal Marginal Relevance)** for diverse results
- **Compression retrievers** for better context
- **Multi-query retrievers** for comprehensive retrieval

### 3. Resources for Learning

**Official Documentation:**
- [Qdrant Documentation](https://qdrant.tech/documentation/)
- [Weaviate Documentation](https://weaviate.io/developers/weaviate)
- [LangChain Documentation](https://python.langchain.com/docs/)
- [Ollama Documentation](https://ollama.ai/)

**LangChain Tutorials:**
- [Vector Stores Guide](https://python.langchain.com/docs/modules/data_connection/vectorstores/)
- [RAG Tutorial](https://python.langchain.com/docs/use_cases/question_answering/)
- [Retrievers Guide](https://python.langchain.com/docs/modules/data_connection/retrievers/)

### 4. Build Your Own Project

Apply what you've learned:
- **Personal knowledge base** - Index your notes and documents
- **Customer support bot** - Search company documentation
- **Code search** - Index and search code repositories
- **Research assistant** - Search academic papers

---

## 🎉 Congratulations!

You now have a solid foundation in vector stores and are ready to build RAG applications!

### Remember:
- Start simple (Qdrant + from_documents)
- Use real embeddings (Ollama)
- Test with small datasets first
- Scale up as needed

### Happy Building! 🚀

---